# Embedding + Bidirectional GRU

In [1]:
import os
import time
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


#### Reading data

In [2]:
dir_data = 'D:/Data_Master/Natural Language Processing/Project1/quora/'
train_df = pd.read_csv(dir_data+'train.csv')
dir_data = 'D:/Data_Master/Natural Language Processing/Project1/quora/'
test_df = pd.read_csv(dir_data+'test.csv')

#### Some preprocessing

Contractions

In [3]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

len(contraction_mapping)

162

In [4]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

Lower

In [5]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())
test_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())

In [6]:
punct_mapping = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping += '©^®` <→°€™› ♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'
puncts = {"‘": "'", "´": "'", "°": "", "€": "e", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '…': ' '}

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    return text


In [7]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_special_chars(x, punct_mapping, puncts))
test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_special_chars(x, punct_mapping, puncts))

#### Tokenizing the words for posterior embedding

In [11]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=123)

## Values of embeddings
embed_size = 300 
max_features = 50000 
maxlen = 100 

train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values


## Shuffling data
np.random.seed(42)

trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
train_y = train_y[trn_idx]
val_X = val_X[val_idx]
val_y = val_y[val_idx]

#### Defining model: Bidirectional GRU + Embeddings

In [12]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140160    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)  

#### Training

In [13]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 835917 samples, validate on 208980 samples
Epoch 1/2


104448/835917 [==>...........................] - ETA: 1:16:48 - loss: 0.6877 - acc: 0.73 - ETA: 53:59 - loss: 0.6746 - acc: 0.8311 - ETA: 46:36 - loss: 0.6610 - acc: 0.87 - ETA: 42:12 - loss: 0.6485 - acc: 0.88 - ETA: 40:22 - loss: 0.6350 - acc: 0.89 - ETA: 38:43 - loss: 0.6221 - acc: 0.89 - ETA: 37:42 - loss: 0.6083 - acc: 0.90 - ETA: 36:41 - loss: 0.5930 - acc: 0.90 - ETA: 36:01 - loss: 0.5794 - acc: 0.91 - ETA: 35:35 - loss: 0.5637 - acc: 0.91 - ETA: 35:06 - loss: 0.5482 - acc: 0.91 - ETA: 34:43 - loss: 0.5330 - acc: 0.91 - ETA: 34:19 - loss: 0.5199 - acc: 0.91 - ETA: 34:06 - loss: 0.5059 - acc: 0.92 - ETA: 33:50 - loss: 0.4934 - acc: 0.92 - ETA: 33:42 - loss: 0.4792 - acc: 0.92 - ETA: 33:29 - loss: 0.4659 - acc: 0.92 - ETA: 33:26 - loss: 0.4537 - acc: 0.92 - ETA: 33:20 - loss: 0.4450 - acc: 0.92 - ETA: 33:14 - loss: 0.4344 - acc: 0.92 - ETA: 33:11 - loss: 0.4267 - acc: 0.92 - ETA: 33:06 - loss: 0.4195 - acc: 0.92 - ETA: 33:02 - loss: 0.4130 - acc: 0.92 - ETA: 32:54 - loss: 0.4060 -

208896/835917 [======>.......................] - ETA: 30:06 - loss: 0.1893 - acc: 0.94 - ETA: 30:04 - loss: 0.1890 - acc: 0.94 - ETA: 30:03 - loss: 0.1886 - acc: 0.94 - ETA: 30:02 - loss: 0.1886 - acc: 0.94 - ETA: 30:01 - loss: 0.1883 - acc: 0.94 - ETA: 30:01 - loss: 0.1881 - acc: 0.94 - ETA: 30:02 - loss: 0.1877 - acc: 0.94 - ETA: 30:01 - loss: 0.1874 - acc: 0.94 - ETA: 30:00 - loss: 0.1871 - acc: 0.94 - ETA: 29:59 - loss: 0.1868 - acc: 0.94 - ETA: 29:58 - loss: 0.1867 - acc: 0.94 - ETA: 29:57 - loss: 0.1864 - acc: 0.94 - ETA: 29:56 - loss: 0.1861 - acc: 0.94 - ETA: 29:55 - loss: 0.1859 - acc: 0.94 - ETA: 29:54 - loss: 0.1857 - acc: 0.94 - ETA: 29:53 - loss: 0.1855 - acc: 0.94 - ETA: 29:52 - loss: 0.1853 - acc: 0.94 - ETA: 29:51 - loss: 0.1851 - acc: 0.94 - ETA: 29:50 - loss: 0.1848 - acc: 0.94 - ETA: 29:48 - loss: 0.1845 - acc: 0.94 - ETA: 29:47 - loss: 0.1845 - acc: 0.94 - ETA: 29:46 - loss: 0.1844 - acc: 0.94 - ETA: 29:45 - loss: 0.1841 - acc: 0.94 - ETA: 29:43 - loss: 0.1838 - acc

313344/835917 [==========>...................] - ETA: 26:32 - loss: 0.1578 - acc: 0.94 - ETA: 26:31 - loss: 0.1577 - acc: 0.94 - ETA: 26:30 - loss: 0.1577 - acc: 0.94 - ETA: 26:30 - loss: 0.1576 - acc: 0.94 - ETA: 26:29 - loss: 0.1576 - acc: 0.94 - ETA: 26:28 - loss: 0.1575 - acc: 0.94 - ETA: 26:27 - loss: 0.1574 - acc: 0.94 - ETA: 26:27 - loss: 0.1573 - acc: 0.94 - ETA: 26:27 - loss: 0.1571 - acc: 0.94 - ETA: 26:26 - loss: 0.1570 - acc: 0.94 - ETA: 26:24 - loss: 0.1569 - acc: 0.94 - ETA: 26:23 - loss: 0.1569 - acc: 0.94 - ETA: 26:22 - loss: 0.1568 - acc: 0.94 - ETA: 26:22 - loss: 0.1567 - acc: 0.94 - ETA: 26:21 - loss: 0.1566 - acc: 0.94 - ETA: 26:20 - loss: 0.1565 - acc: 0.94 - ETA: 26:18 - loss: 0.1564 - acc: 0.94 - ETA: 26:17 - loss: 0.1563 - acc: 0.94 - ETA: 26:16 - loss: 0.1563 - acc: 0.94 - ETA: 26:15 - loss: 0.1563 - acc: 0.94 - ETA: 26:13 - loss: 0.1562 - acc: 0.94 - ETA: 26:12 - loss: 0.1561 - acc: 0.94 - ETA: 26:11 - loss: 0.1560 - acc: 0.94 - ETA: 26:10 - loss: 0.1560 - acc

417792/835917 [=============>................] - ETA: 22:27 - loss: 0.1445 - acc: 0.94 - ETA: 22:26 - loss: 0.1444 - acc: 0.94 - ETA: 22:24 - loss: 0.1444 - acc: 0.94 - ETA: 22:23 - loss: 0.1444 - acc: 0.94 - ETA: 22:21 - loss: 0.1444 - acc: 0.94 - ETA: 22:21 - loss: 0.1443 - acc: 0.94 - ETA: 22:19 - loss: 0.1443 - acc: 0.94 - ETA: 22:18 - loss: 0.1442 - acc: 0.94 - ETA: 22:17 - loss: 0.1442 - acc: 0.94 - ETA: 22:15 - loss: 0.1442 - acc: 0.94 - ETA: 22:14 - loss: 0.1442 - acc: 0.94 - ETA: 22:13 - loss: 0.1441 - acc: 0.94 - ETA: 22:11 - loss: 0.1440 - acc: 0.94 - ETA: 22:10 - loss: 0.1439 - acc: 0.94 - ETA: 22:09 - loss: 0.1439 - acc: 0.94 - ETA: 22:08 - loss: 0.1438 - acc: 0.94 - ETA: 22:06 - loss: 0.1438 - acc: 0.94 - ETA: 22:05 - loss: 0.1438 - acc: 0.94 - ETA: 22:04 - loss: 0.1437 - acc: 0.94 - ETA: 22:02 - loss: 0.1436 - acc: 0.94 - ETA: 22:01 - loss: 0.1436 - acc: 0.94 - ETA: 22:00 - loss: 0.1436 - acc: 0.94 - ETA: 21:59 - loss: 0.1436 - acc: 0.94 - ETA: 21:58 - loss: 0.1435 - acc

...

417792/835917 [=============>................] - ETA: 21:41 - loss: 0.0978 - acc: 0.96 - ETA: 21:40 - loss: 0.0979 - acc: 0.96 - ETA: 21:39 - loss: 0.0978 - acc: 0.96 - ETA: 21:38 - loss: 0.0978 - acc: 0.96 - ETA: 21:36 - loss: 0.0977 - acc: 0.96 - ETA: 21:35 - loss: 0.0978 - acc: 0.96 - ETA: 21:34 - loss: 0.0978 - acc: 0.96 - ETA: 21:32 - loss: 0.0978 - acc: 0.96 - ETA: 21:31 - loss: 0.0978 - acc: 0.96 - ETA: 21:30 - loss: 0.0978 - acc: 0.96 - ETA: 21:29 - loss: 0.0978 - acc: 0.96 - ETA: 21:27 - loss: 0.0978 - acc: 0.96 - ETA: 21:26 - loss: 0.0978 - acc: 0.96 - ETA: 21:25 - loss: 0.0978 - acc: 0.96 - ETA: 21:24 - loss: 0.0978 - acc: 0.96 - ETA: 21:22 - loss: 0.0978 - acc: 0.96 - ETA: 21:21 - loss: 0.0978 - acc: 0.96 - ETA: 21:20 - loss: 0.0978 - acc: 0.96 - ETA: 21:18 - loss: 0.0978 - acc: 0.96 - ETA: 21:17 - loss: 0.0978 - acc: 0.96 - ETA: 21:16 - loss: 0.0978 - acc: 0.96 - ETA: 21:15 - loss: 0.0978 - acc: 0.96 - ETA: 21:13 - loss: 0.0979 - acc: 0.96 - ETA: 21:12 - loss: 0.0979 - acc

522240/835917 [=================>............] - ETA: 17:21 - loss: 0.0984 - acc: 0.96 - ETA: 17:20 - loss: 0.0984 - acc: 0.96 - ETA: 17:19 - loss: 0.0984 - acc: 0.96 - ETA: 17:17 - loss: 0.0984 - acc: 0.96 - ETA: 17:16 - loss: 0.0984 - acc: 0.96 - ETA: 17:15 - loss: 0.0985 - acc: 0.96 - ETA: 17:14 - loss: 0.0985 - acc: 0.96 - ETA: 17:12 - loss: 0.0985 - acc: 0.96 - ETA: 17:11 - loss: 0.0985 - acc: 0.96 - ETA: 17:10 - loss: 0.0985 - acc: 0.96 - ETA: 17:09 - loss: 0.0985 - acc: 0.96 - ETA: 17:07 - loss: 0.0985 - acc: 0.96 - ETA: 17:06 - loss: 0.0985 - acc: 0.96 - ETA: 17:05 - loss: 0.0985 - acc: 0.96 - ETA: 17:03 - loss: 0.0985 - acc: 0.96 - ETA: 17:02 - loss: 0.0985 - acc: 0.96 - ETA: 17:01 - loss: 0.0985 - acc: 0.96 - ETA: 17:00 - loss: 0.0985 - acc: 0.96 - ETA: 16:58 - loss: 0.0985 - acc: 0.96 - ETA: 16:57 - loss: 0.0985 - acc: 0.96 - ETA: 16:56 - loss: 0.0985 - acc: 0.96 - ETA: 16:54 - loss: 0.0985 - acc: 0.96 - ETA: 16:53 - loss: 0.0985 - acc: 0.96 - ETA: 16:52 - loss: 0.0985 - acc

626688/835917 [=====================>........] - ETA: 13:00 - loss: 0.0985 - acc: 0.96 - ETA: 12:59 - loss: 0.0985 - acc: 0.96 - ETA: 12:58 - loss: 0.0985 - acc: 0.96 - ETA: 12:57 - loss: 0.0985 - acc: 0.96 - ETA: 12:55 - loss: 0.0985 - acc: 0.96 - ETA: 12:54 - loss: 0.0985 - acc: 0.96 - ETA: 12:53 - loss: 0.0985 - acc: 0.96 - ETA: 12:51 - loss: 0.0985 - acc: 0.96 - ETA: 12:50 - loss: 0.0985 - acc: 0.96 - ETA: 12:49 - loss: 0.0985 - acc: 0.96 - ETA: 12:48 - loss: 0.0985 - acc: 0.96 - ETA: 12:46 - loss: 0.0985 - acc: 0.96 - ETA: 12:45 - loss: 0.0985 - acc: 0.96 - ETA: 12:44 - loss: 0.0985 - acc: 0.96 - ETA: 12:42 - loss: 0.0985 - acc: 0.96 - ETA: 12:41 - loss: 0.0985 - acc: 0.96 - ETA: 12:40 - loss: 0.0985 - acc: 0.96 - ETA: 12:39 - loss: 0.0985 - acc: 0.96 - ETA: 12:37 - loss: 0.0985 - acc: 0.96 - ETA: 12:36 - loss: 0.0986 - acc: 0.96 - ETA: 12:35 - loss: 0.0986 - acc: 0.96 - ETA: 12:34 - loss: 0.0986 - acc: 0.96 - ETA: 12:32 - loss: 0.0986 - acc: 0.96 - ETA: 12:31 - loss: 0.0986 - acc

731136/835917 [=========================>....] - ETA: 8:40 - loss: 0.0988 - acc: 0.960 - ETA: 8:39 - loss: 0.0988 - acc: 0.960 - ETA: 8:37 - loss: 0.0988 - acc: 0.960 - ETA: 8:36 - loss: 0.0988 - acc: 0.960 - ETA: 8:35 - loss: 0.0988 - acc: 0.960 - ETA: 8:34 - loss: 0.0988 - acc: 0.960 - ETA: 8:32 - loss: 0.0988 - acc: 0.960 - ETA: 8:31 - loss: 0.0988 - acc: 0.960 - ETA: 8:30 - loss: 0.0988 - acc: 0.960 - ETA: 8:29 - loss: 0.0988 - acc: 0.960 - ETA: 8:27 - loss: 0.0988 - acc: 0.960 - ETA: 8:26 - loss: 0.0988 - acc: 0.960 - ETA: 8:25 - loss: 0.0988 - acc: 0.960 - ETA: 8:23 - loss: 0.0988 - acc: 0.960 - ETA: 8:22 - loss: 0.0989 - acc: 0.960 - ETA: 8:21 - loss: 0.0989 - acc: 0.960 - ETA: 8:20 - loss: 0.0989 - acc: 0.960 - ETA: 8:18 - loss: 0.0989 - acc: 0.960 - ETA: 8:17 - loss: 0.0989 - acc: 0.960 - ETA: 8:16 - loss: 0.0989 - acc: 0.960 - ETA: 8:14 - loss: 0.0989 - acc: 0.960 - ETA: 8:13 - loss: 0.0989 - acc: 0.960 - ETA: 8:12 - loss: 0.0989 - acc: 0.960 - ETA: 8:11 - loss: 0.0989 - acc:

835917/835917 [==============================] - ETA: 4:20 - loss: 0.0988 - acc: 0.960 - ETA: 4:18 - loss: 0.0988 - acc: 0.960 - ETA: 4:17 - loss: 0.0988 - acc: 0.960 - ETA: 4:16 - loss: 0.0988 - acc: 0.960 - ETA: 4:14 - loss: 0.0989 - acc: 0.960 - ETA: 4:13 - loss: 0.0989 - acc: 0.960 - ETA: 4:12 - loss: 0.0989 - acc: 0.960 - ETA: 4:11 - loss: 0.0989 - acc: 0.960 - ETA: 4:09 - loss: 0.0989 - acc: 0.960 - ETA: 4:08 - loss: 0.0989 - acc: 0.960 - ETA: 4:07 - loss: 0.0989 - acc: 0.960 - ETA: 4:05 - loss: 0.0989 - acc: 0.960 - ETA: 4:04 - loss: 0.0989 - acc: 0.960 - ETA: 4:03 - loss: 0.0989 - acc: 0.960 - ETA: 4:02 - loss: 0.0989 - acc: 0.960 - ETA: 4:00 - loss: 0.0988 - acc: 0.960 - ETA: 3:59 - loss: 0.0988 - acc: 0.960 - ETA: 3:58 - loss: 0.0988 - acc: 0.960 - ETA: 3:57 - loss: 0.0988 - acc: 0.960 - ETA: 3:55 - loss: 0.0988 - acc: 0.960 - ETA: 3:54 - loss: 0.0988 - acc: 0.960 - ETA: 3:53 - loss: 0.0988 - acc: 0.960 - ETA: 3:51 - loss: 0.0988 - acc: 0.960 - ETA: 3:50 - loss: 0.0988 - acc:

In [ ]:
# model.save_weights("model_Embeddings+GRU.h5")

#### Fscore for validation with different thresholds

In [ ]:
pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)

 83968/208980 [===========>..................] - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1:36

In [23]:
f1_final = 0
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    f1 = metrics.f1_score(val_y, (pred_val_y>threshold).astype(int))
    if f1>f1_final:
        f1_final = f1
        threshold = thresh
    
print("F1 score at threshold {0} is {1}".format(threshold, metrics.f1_score(val_y, (pred_val_y>threshold).astype(int))))

F1 score at threshold 0.27 is 0.6471180106367688


# Testing with real data test

In [ ]:
pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
pred_test_y = (pred_test_y>threshold).astype(int)

In [ ]:
submission_df = pd.DataFrame({"qid":test_df["qid"].values})
submission_df['prediction'] = pred_test_y
submission_df.to_csv("submission.csv", index=False)